In [ ]:
import os
from openai import OpenAI

openai_api_key = os.environ.get('OPENAI_API_KEY')
deepgram_api_key = os.environ.get('DEEPGRAM_API_KEY')


In [ ]:

client = OpenAI()
def createAssistant():
    assistant = client.beta.assistants.create(
        name = "Mental Health Assistant", 
        instructions = "You are an assistant who is generating prompts to help a professional in talking to someone with mental health concerns. Generate potential responses for what the professional could say/ask", 
        tools = [{"type":"file_search"}],
        model = "gpt-4o",
    )
    return assistant

def uploadFiles(assistantname, files):
    vector_store = client.beta.vector_stores.create(name="Mental Health Documents")
    file_paths = files
    file_streams = [open(path, "rb") for path in file_paths]
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams)
    assistant = client.beta.assistants.update(
        assistant_id=assistantname.id, tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
    )

def openCoversation():
    thread = client.beta.threads.create()
    return thread

def askQuestion(thread, query):
    message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = query
    )



In [ ]:

assistant = createAssistant()
conversation = openCoversation()
uploadFiles(assistant, ["SUICIDE_AWARENESS_PREVENTION_TRAINING.pdf"])


askQuestion(conversation, input())

run = client.beta.threads.runs.create_and_poll(
thread_id = conversation.id,
assistant_id = assistant.id,
instructions = "Answer the questions in a brief 15 words. The input you are given is a response from someone with mental health issues. Suggest some responses that could be said, using the uploaded files to aid your answer"
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=conversation.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)



    

Mental health training often covers recognizing signs, asking questions, validating experiences, and encouraging treatment【4:2†SUICIDE_AWARENESS_PREVENTION_TRAINING.pdf】.
